In [1]:
import pandas as pd
import gzip
import json
import numpy as np
import math
from random import randint



In [ ]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Digital_Music.json.gz')

In [ ]:
df_clean = df.drop(columns=['reviewTime', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'style', 'image', 'vote', 'verified'])

In [ ]:
#Filtering the data by number of reviews before making pivot table to ease memory constraints
#Here threshold is inclusive (n or more reviews will be kept)
user_thresh = 5
item_thresh = 3
pre_pivot = df_clean[df_clean.groupby(['reviewerID'])['reviewerID'].transform('size') >= user_thresh]
pre_pivot2 = pre_pivot[pre_pivot.groupby(['asin'])['asin'].transform('size') >= item_thresh]

In [ ]:
full_dict = {}

In [ ]:
for index, row in pre_pivot2.iterrows():
    if row['reviewerID'] in full_dict: 
        full_dict[row['reviewerID']].append(([row['asin'], row['overall']]))
    else:
        full_dict[row['reviewerID']] = ([[row['asin'], row['overall']]])

In [ ]:
full_dict['AJO3UG6FR5C7R']

In [ ]:
len(full_dict)

In [ ]:
pruned_dict = {}
for key in full_dict:
    if len(full_dict[key]) >= 5:
        pruned_dict[key] = full_dict[key]
len(pruned_dict)

In [ ]:
list(pruned_dict.items())[:10]

In [ ]:
test_dict = {}
for key in pruned_dict:
    take = math.floor(len(pruned_dict[key])/5)
    for i in range(take):
        rand = randint(0, len(pruned_dict[key]) - 1)
        if key in test_dict: 
            test_dict[key].append(pruned_dict[key].pop(rand))
        else:
            test_dict[key] = ([pruned_dict[key].pop(rand)])
        

In [ ]:
train_dict = pruned_dict

In [ ]:
len(test_dict)

In [ ]:
list(train_dict.items())[:10]

In [ ]:
list(test_dict.items())[:10]